# Basic example of simulation on Torus

In this example, we run a Navground simulation on a Torus defined by a square with periodic boundary conditions. We explore the basics of computing persistence divergence diagrams.

We start by importing a few libraries. We use the `perdiver` library, which in turn uses the `tdqual` library for computing the matching diagrams.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist
import matplotlib as mpl

plots_dir = "plots/"
import os

os.makedirs(plots_dir, exist_ok=True)

from perdiver.distances import *
import perdiver.perdiver as perdiver

We run a simulation with `10`agents using the ORCA behaviour.

In [ ]:
from navground import sim, core
side=20
# behaviour = "ORCA"
# # The following YAML contains the information for running the Navground simulation.
# yaml = f"""
# steps: 1000
# time_step: 0.1
# record_pose: true
# record_twist: true
# runs: 1
# scenario:
#   type: CrossTorus
#   agent_margin: 0.2
#   side: {side}
#   target_margin: 0.6
#   tolerance: 0.4
#   groups:   
#     -
#       type: thymio
#       number: 10
#       control_period: 0.1
#       behavior:
#         type: {behaviour}
#         safety_margin: 0.2 
#         horizon: 3
#         barrier_angle: 1
#       radius: 0.2
#       kinematics:
#         type: 2WDiff
#         max_speed: 1.0
#         wheel_axis: 2
#       state_estimation:
#         type: Bounded
#         range: 2.0 
# """
# experiment = sim.load_experiment(yaml)
path = "recorded_simulation.h5"
# experiment.run(keep=False, data_path=path)
# We load the coordinates and velocities at all timesteps 
recorded_experiment = sim.RecordedExperiment(path)
run = recorded_experiment.runs[0]
ps = run.poses[:,:,[0,1]]
twists = run.twists[:,:,:2] # ignore angular speeds

Now, we plot two timesteps with their velocities.

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
start=110
end=140
perdiver.plot_two_timesteps_with_velocities(ps[start], ps[end], twists[start], twists[end], ax, X_col="blue", Y_col="red")
ax.set_xticks([])
ax.set_yticks([])
plt.savefig(f"{plots_dir}two_timesteps.png")

Next, compute distance matrix.

In [ ]:
weight=1
fig, ax = plt.subplots(ncols=2, figsize=(7,3.5))
for idx, w in zip(range(2), [1,5]):
    Dist_X, Dist_Y, Dist_Z = compute_distance_matrices_timesteps_2Dtorus(ps[start], ps[end], w*twists[start], w*twists[end], weight, side)
    match_diagram = perdiver.get_matching_diagram(Dist_X, Dist_Y)
    perdiver.plot_matching_diagram(match_diagram, ax[idx], color="blue")
    ax[idx].set_title(f"D(f) for weight {w:2d}")

plt.savefig(f"{plots_dir}two_timesteps_matching_diagram.png")